In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
# import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
# torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)

#### The Data

In [19]:
# train_df = pd.read_csv('../../final/data/train_set_basic.csv')
train_df = pd.read_csv('../../../anemia_ml4hc/data/train_set_noisiness_0.1.csv')
#train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.302918,140.296774,-1.000000,0.797081,381.210081,104.827986,-1.000000,2.089972,0,1.165053,84.720406,-1.000000,-1.000000,-1.000000,-1.000000,21.908754,-1.000000,1.0
1,7.410904,153.255383,-1.000000,-1.000000,-1.000000,77.015598,-1.000000,2.886780,0,1.880117,28.321287,-1.000000,-1.000000,-1.000000,-1.000000,22.232711,-1.000000,3.0
2,7.741627,310.690809,4.962075,-1.000000,152.546597,78.879642,79.242201,2.944344,0,1.155946,67.896356,-1.000000,31.326490,-1.000000,76.923925,23.224880,51.946226,3.0
3,7.663978,103.521761,-1.000000,-1.000000,376.479031,79.326095,-1.000000,2.898408,1,1.320781,76.086364,-1.000000,-1.000000,-1.000000,-1.000000,22.991935,-1.000000,0.0
4,7.546320,250.950487,1.983130,6.982427,307.773489,91.378483,20.361380,2.477493,1,1.900588,134.946814,72.665106,63.584247,21.537545,57.128744,22.638960,6.615703,5.0


In [20]:
train_df.label.value_counts()

0.0    11520
6.0     5858
2.0     5841
5.0     5833
1.0     5831
3.0     5714
4.0     5433
7.0     4370
Name: label, dtype: int64

In [21]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [22]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50400, 17), (14000, 17), (50400,), (14000,))

#### Training 

In [15]:
# %%time
# timesteps = int(1e6)
# dqn_model = utils.stable_dqn(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
# for steps in [int(50e6)]:
#     dqn_model = utils.stable_dqn(X_train, y_train, steps, True, 
#                                  f'../../../anemia_ml4hc/models/complete_models/dueling_ddqn_{steps}',
#                                  checkpoint_folder = '../../../anemia_ml4hc/models/sb/dueling_double_dqn',
#                                  checkpoint_prefix='dddqn_basic')
    
for steps in [int(15e6), int(12e6), int(10e6), int(7e6), int(5e6)]:
# for steps in [int(2e3)]:
    dqn_model = utils.stable_double_dqn(X_train, y_train, steps, True, 
                                 f'../../../anemia_ml4hc/models/sb/missingness/double_dqn_pr_noisiness_1_{steps}')#,
#                                  checkpoint_folder = '../../../anemia_ml4hc/models/sb/dueling_double_dqn',
#                                  checkpoint_prefix='dqn_basic')

using just stable baselines (not 3)
--------------------------------------
| % time spent exploring  | 98       |
| episodes                | 10000    |
| mean 100 episode reward | -0.8     |
| steps                   | 26522    |
| success rate            | 0.1      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 96       |
| episodes                | 20000    |
| mean 100 episode reward | -0.7     |
| steps                   | 52734    |
| success rate            | 0.17     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 94       |
| episodes                | 30000    |
| mean 100 episode reward | -0.6     |
| steps                   | 80091    |
| success rate            | 0.18     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 93       |
| episodes                | 40000    |
| mean 100 episode reward | 

--------------------------------------
| % time spent exploring  | 39       |
| episodes                | 310000   |
| mean 100 episode reward | -0.9     |
| steps                   | 953704   |
| success rate            | 0.12     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 37       |
| episodes                | 320000   |
| mean 100 episode reward | -0.9     |
| steps                   | 990931   |
| success rate            | 0.07     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 35       |
| episodes                | 330000   |
| mean 100 episode reward | -0.9     |
| steps                   | 1025262  |
| success rate            | 0.08     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 32       |
| episodes                | 340000   |
| mean 100 episode reward | -0.8     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 620000   |
| mean 100 episode reward | -0.5     |
| steps                   | 1948833  |
| success rate            | 0.25     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 630000   |
| mean 100 episode reward | -0.3     |
| steps                   | 1982028  |
| success rate            | 0.41     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 640000   |
| mean 100 episode reward | -0.4     |
| steps                   | 2016022  |
| success rate            | 0.34     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 650000   |
| mean 100 episode reward | -0.6     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 930000   |
| mean 100 episode reward | -0.2     |
| steps                   | 3053389  |
| success rate            | 0.43     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 940000   |
| mean 100 episode reward | -0       |
| steps                   | 3089300  |
| success rate            | 0.48     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 950000   |
| mean 100 episode reward | -0.3     |
| steps                   | 3125500  |
| success rate            | 0.36     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 960000   |
| mean 100 episode reward | -0.3     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1240000  |
| mean 100 episode reward | -0       |
| steps                   | 4223303  |
| success rate            | 0.5      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1250000  |
| mean 100 episode reward | -0.1     |
| steps                   | 4263659  |
| success rate            | 0.43     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1260000  |
| mean 100 episode reward | -0.2     |
| steps                   | 4303933  |
| success rate            | 0.43     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1270000  |
| mean 100 episode reward | -0.2     |
| steps                  

...

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1480000  |
| mean 100 episode reward | -0.1     |
| steps                   | 5538766  |
| success rate            | 0.53     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1490000  |
| mean 100 episode reward | 0        |
| steps                   | 5582875  |
| success rate            | 0.58     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1500000  |
| mean 100 episode reward | 0        |
| steps                   | 5625469  |
| success rate            | 0.58     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1510000  |
| mean 100 episode reward | 0        |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1790000  |
| mean 100 episode reward | -0.1     |
| steps                   | 6941755  |
| success rate            | 0.55     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1800000  |
| mean 100 episode reward | 0.3      |
| steps                   | 6985405  |
| success rate            | 0.7      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1810000  |
| mean 100 episode reward | 0.2      |
| steps                   | 7031373  |
| success rate            | 0.69     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1820000  |
| mean 100 episode reward | 0.1      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2100000  |
| mean 100 episode reward | -0       |
| steps                   | 8329882  |
| success rate            | 0.56     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2110000  |
| mean 100 episode reward | 0.1      |
| steps                   | 8374660  |
| success rate            | 0.65     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2120000  |
| mean 100 episode reward | -0       |
| steps                   | 8417777  |
| success rate            | 0.59     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2130000  |
| mean 100 episode reward | 0.1      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2410000  |
| mean 100 episode reward | 0.2      |
| steps                   | 9725596  |
| success rate            | 0.64     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2420000  |
| mean 100 episode reward | 0        |
| steps                   | 9770282  |
| success rate            | 0.64     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2430000  |
| mean 100 episode reward | -0       |
| steps                   | 9815696  |
| success rate            | 0.57     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2440000  |
| mean 100 episode reward | 0.1      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 240000   |
| mean 100 episode reward | -0.7     |
| steps                   | 820177   |
| success rate            | 0.19     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 250000   |
| mean 100 episode reward | -0.6     |
| steps                   | 852459   |
| success rate            | 0.22     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 260000   |
| mean 100 episode reward | -0.5     |
| steps                   | 885084   |
| success rate            | 0.24     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 270000   |
| mean 100 episode reward | -0.5     |
| steps                  

#### Testing

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/test_df_2800000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/success_df_2800000.csv', index=False)